# Creación de ImageSet 

In [ ]:
from CGRfuncs import data_set_gen

Generación de conjunto de validación. Se almacena en `data/filtered/vanilla_set/test`

In [ ]:
data_set_gen('../data/raw_data/test/alpha_test.fasta',
             '../data/filtered/vanilla_set/test/alpha', 'alpha')

data_set_gen('../data/raw_data/test/beta_test.fasta',
             '../data/filtered/vanilla_set/test/beta', 'beta')

data_set_gen('../data/raw_data/test/delta_test.fasta',
             '../data/filtered/vanilla_set/test/delta', 'delta')

data_set_gen('../data/raw_data/test/gamma_test.fasta',
             '../data/filtered/vanilla_set/test/gamma', 'gamma')

Generación de conjunto de validación. Se almacena en `data/filtered/vanilla_set/train`

In [ ]:
data_set_gen('../data/raw_data/train/alpha_train.fasta',
             '../data/filtered/vanilla_set/train/alpha', 'alpha')

data_set_gen('../data/raw_data/train/beta_train.fasta',
             '../data/filtered/vanilla_set/train/beta', 'beta')

data_set_gen('../data/raw_data/train/delta_train.fasta',
             '../data/filtered/vanilla_set/train/delta', 'delta')

data_set_gen('../data/raw_data/train/gamma_train.fasta',
             '../data/filtered/vanilla_set/train/gamma', 'gamma')

# Creacion de ImageSet ruidoso en caso de desearlo

Agrega ruido a los conjuntos de prueba

In [ ]:
noise_data_gen('../data/filtered/noised_set/test/alpha', '../data/filtered/noised_set/test/alpha',
                50, 0, 'alpha_noise')

noise_data_gen('../data/filtered/noised_set/test/beta', '../data/filtered/noised_set/test/beta',
                50, 0, 'beta_noise')

noise_data_gen('../data/filtered/noised_set/test/delta', '../data/filtered/noised_set/test/delta',
               82, 0, 'delta_noise')

noise_data_gen('../data/filtered/noised_set/test/gamma', '../data/filtered/noised_set/test/gamma',
               50, 0, 'gamma_noise')

Agrega ruido a los conjuntos de entrenamiento

In [ ]:
noise_data_gen('../data/filtered/noised_set/train/alpha', '../data/filtered/noised_set/train/alpha',
               100, 0, 'alpha_noise')

noise_data_gen('../data/filtered/noised_set/train/beta', '../data/filtered/noised_set/train/beta',
               100, 0, 'beta_noise')

noise_data_gen('../data/filtered/noised_set/train/delta', '../data/filtered/noised_set/train/delta',
              176, 0, 'delta_noise')

noise_data_gen('../data/filtered/noised_set/train/gamma', '../data/filtered/noised_set/train/gamma',
              100, 0, 'gamma_noise')

# Creación de Modelo

In [ ]:
from Model import BaseVGG

Por default se usará el conjunto sin ruido

In [ ]:
m = BaseVGG(200,200)
m.load_data('../data/filtered/vanilla_set/train', 
            '../data/filtered/vanilla_set/test')
m.train(200, 100, '../archive/nombre-del-test', 'numeroTest')

# Predicción de imágenes a partir del modelo guardado

In [ ]:
import cv2, pickle

Generación de imágenes para predección

In [ ]:
data_set_gen('../data/raw_data/bluga_whale_coronavirus_SW1_gammacoronavirus.fasta', 
             '../data/filtered/to_predict', 'beluga_gamma')
data_set_gen('../data/raw_data/HCoV-229E_alphacoronavirus.fasta', 
             '../data/filtered/to_predict', 'hcov_alpha')
data_set_gen('../data/raw_data/porcine_diarrhea_CoV_deltacoronavirus.fasta', 
             '../data/filtered/to_predict', 'procine_delta')
data_set_gen('../data/raw_data/SARS-CoV2_betacoronavirus.fasta', 
             '../data/filtered/to_predict', 'sarscov2_beta')

In [ ]:
im = cv2.imread('../data/filtered/to_predict/nombre-de-imagen', cv2.IMREAD_GRAYSCALE)
resized = cv2.resize(im, (200,200), interpolation=cv2.INTER_AREA)

In [ ]:
# Carga el modelo completo 
predictor = pickle.load(open('../archive/directorio-test/nombre-test.sav', 'rb'))

In [ ]:
print(predictor.predict(im.reshape(1,200,200,1)))